In [1]:
pip show openai


Name: openai
Version: 0.28.0
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: p:\generative_ai\venv\lib\site-packages
Requires: aiohttp, requests, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
import openai
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = "voc-605570984126677350407866e29880939962.25846762"
# Load the dataset
df = pd.read_csv('nyc_food_scrap_drop_off_sites.csv')
df.head()


,Borough,NTAName,SiteName,SiteAddr,Hosted_By,Open_Month,Day_Hours,Notes,Website,BoroCD,CouncilDis,ct2010,BBL,BIN,Latitude,Longitude,PolicePrec,Object ID,Location Point
0,Manhattan,Inwood,SW West 212th Street & 10th Avenue,NaN,Department of Sanitation,Year Round,24/7,Download the app to access bins. Accepts all f...,www.nyc.gov/smartcomposting,112,10,NaN,NaN,NaN,NaN,NaN,34,6914,NaN
1,Brooklyn,Park Slope,Old Stone House Brooklyn,"336 3rd St, Brooklyn, NY 11215",Old Stone House Brooklyn,Year Round,24/7 (Start Time: 24/7 - End Time: 24/7),NaN,NaN,306,39,NaN,NaN,NaN,40.672712,-73.984731,78,6777,POINT (-73.984731 40.6727118)
2,Bronx,Norwood,NE Hull Avenue & East 207th Street,NaN,Department of Sanitation,Year Round,24/7,Download the app to access bins. Accepts all f...,www.nyc.gov/smartcomposting,207,11,NaN,NaN,NaN,NaN,NaN,52,7115,NaN
3,Manhattan,East Harlem (South),SE East 112th Street & Park Avenue,NaN,Department of Sanitation,Year Round,24/7,Download the app to access bins. Accepts all f...,www.nyc.gov/smartcomposting,111,8,NaN,NaN,NaN,NaN,NaN,23,6819,NaN
4,Queens,Astoria (Central),Astoria Pug: Broadway,"32-11 31st St, Queens, NY 11106",Astoria Pug,Year Round,Saturdays (Start Time: 9:00 AM - End Time: 3:...,"Not accepted: meat, bones, or dairy",https://www.instagram.com/astoriapug/?hl=en,401,22,NaN,NaN,NaN,40.761472,-73.925611,114,7046,POINT (-73.925611 40.761472)


In [34]:
# Remove unnecessary columns
df_cleaned = df[['Borough', 'SiteName', 'SiteAddr', 'Day_Hours', 'Notes']].dropna()
# Further cleaning if required (e.g., removing duplicates, trimming whitespace)
df_cleaned['SiteAddr'] = df_cleaned['SiteAddr'].str.strip()
df_cleaned = df_cleaned.drop_duplicates()
# Display the cleaned dataframe
df_cleaned.head()

,Borough,SiteName,SiteAddr,Day_Hours,Notes
4,Queens,Astoria Pug: Broadway,"32-11 31st St, Queens, NY 11106",Saturdays (Start Time: 9:00 AM - End Time: 3:...,"Not accepted: meat, bones, or dairy"
5,Brooklyn,East New York Farms: St. John Community Garden,"474 New Jersey Ave, Brooklyn, NY 11207",24/7 (Start Time: 24/7 - End Time: 24/7),"Not accepted: meat, bones, or dairy"
9,Manhattan,Battery Park City Authority Rockefeller Park,Lower level downstairs at corner of Chambers S...,24/7 (Start Time: 24/7 - End Time: 24/7),"Not accepted: meat, bones, or dairy"
12,Bronx,Drew Gardens,1070 East Tremont Avenue. between Bronx Street...,"Mondays, Wednesdays, and Fridays (Start Time: ...","Not accepted: meat, bones, or dairy"
20,Staten Island,*CLOSED FOR THE SEASON* West Brighton,Chappell Street and Henderson Avenue,Fridays (Start Time: 4:00 PM - End Time: 5:30...,"Not accepted: meat, bones, or dairy"


In [21]:
# Function to handle chatbot queries
def chatbot_query(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.7,
    )
    # print(response)
    return response.choices[0].message.content.strip()


In [22]:
# Example user interaction
user_input = "Where can I drop off food scraps in Brooklyn?"
bot_response = chatbot_query(user_input)
print(bot_response)

You can drop off food scraps for composting at various locations in Brooklyn. Some options include community compost drop-off sites, farmers markets, and select greenmarkets. You can also check with your local community garden or composting organization for drop-off locations near you. Additionally, some residential buildings and local businesses offer composting services. It's always a good idea to call ahead or check online for the most up-to-date information on drop-off locations in Brooklyn.


In [35]:
# Function to filter locations based on borough
def search_drop_off_location(borough):
    # Filter based on borough
    results = df_cleaned[df_cleaned['Borough'].str.contains(borough, case=False, na=False)]
    return results[['SiteName', 'SiteAddr', 'Day_Hours', 'Notes']]

user_borough = "Brooklyn"
locations = search_drop_off_location(user_borough)

# Display top 5 results for simplicity
print(locations.head())


                                          SiteName  \
5   East New York Farms: St. John Community Garden   
27                       Transmitter Park Drop-Off   
37                         Red Hook Community Farm   
40                                     Bensonhurst   
48                            Rappaport Playground   

                                           SiteAddr  \
5            474 New Jersey Ave, Brooklyn, NY 11207   
27                 2 Greenpoint Ave and West Street   
37                                    103 Otsego St   
40                18th Ave btwn 81st & 82nd Streets   
48  NW corner of Fort Hamilton Pkwy and 53rd Street   

                                            Day_Hours  \
5           24/7 (Start Time: 24/7 - End Time:  24/7)   
27  Sundays (Start Time: 11:30 AM - End Time:  1:0...   
37          24/7 (Start Time: 24/7 - End Time:  24/7)   
40  Sundays (Start Time: 9:00 AM - End Time:  1:00...   
48  Tuesdays (Start Time: 10:00 AM - End Time:  2:...   

 

In [37]:
# def chatbot_with_data(query, borough):
#     # Get AI-generated response
#     ai_response = chatbot_query(query)
    
#     # Filter data based on borough
#     borough_data = search_drop_off_location(borough)

#     print(borough_data)
#     # Format the response
#     response = f"{ai_response}\nHere are some drop-off locations in {borough}:\n{borough_data['NTAName'].head().to_string(index=False)}"
#     return response

# # Example interaction
# user_query = "Where can I drop off food scraps in Brooklyn?"
# print(chatbot_with_data(user_query, "Brooklyn"))


In [ ]:
# Continuous interaction using a while loop
def start_chatbot():
    print("Welcome to the NYC Food Scrap Drop-Off chatbot! Type 'exit' to end the conversation.")
    while True:
        # Ask the user for a question
        user_input = input("Ask a question: ")
        
        # Break the loop if the user wants to exit
        if user_input.lower() == "exit":
            print("Goodbye!")
            break

        # Filter the data based on a potential borough mentioned by the user
        borough = None
        if 'brooklyn' in user_input.lower():
            borough = 'Brooklyn'
        elif 'manhattan' in user_input.lower():
            borough = 'Manhattan'
        elif 'queens' in user_input.lower():
            borough = 'Queens'
        elif 'bronx' in user_input.lower():
            borough = 'Bronx'
        elif 'staten island' in user_input.lower():
            borough = 'Staten Island'

        # Provide responses using both the chatbot and data if a borough is mentioned
        if borough:
            borough_data = search_drop_off_location(borough)
            if not borough_data.empty:
                # Get AI-generated response
                ai_response = chatbot_query(user_input)
                # Display chatbot response and borough-specific locations
                print(f"{ai_response}\nHere are some drop-off locations in {borough}:\n{borough_data.to_string(index=False)}")
            else:
                print(f"Sorry, I couldn't find any locations for {borough}.")
        else:
            # If no borough is mentioned, just give a chatbot-generated response
            bot_response = chatbot_query(user_input)
            print(bot_response)

# Start the chatbot
start_chatbot()

Welcome to the NYC Food Scrap Drop-Off chatbot! Type 'exit' to end the conversation.


Ask a question:  where can i drop off ?


Drop off what? Can you please provide more context so I can assist you better?


Ask a question:  i am talking about food scrap!


That's great! Food scraps can be recycled and used to create compost, which is beneficial for the environment. Composting food scraps helps reduce waste sent to landfills and can be used to improve soil quality in gardens and farms. If you have any specific questions about food scraps or composting, feel free to ask!


...

I would be happy to help you, but I need a bit more information. What are you looking to drop off?


Ask a question:  food scrap


Food scraps are organic materials left over from cooking or eating, such as fruit and vegetable peels, coffee grounds, eggshells, and leftover food. Instead of throwing them away, food scraps can be composted to create nutrient-rich soil for gardening. Composting food scraps helps reduce waste and benefits the environment by diverting organic materials from landfills. If you're interested in composting your food scraps, I can provide you with tips on how to get started.


Ask a question:  Where can I drop off food scraps in Brooklyn?


You can drop off food scraps for composting at various locations in Brooklyn. Some options include Greenmarkets, community gardens, and certain Whole Foods locations. You can visit the NYC Compost Project website or the GrowNYC website for more information on drop-off locations and hours of operation.
Here are some drop-off locations in Brooklyn:
                                                                  SiteName                                                                                                        SiteAddr                                                                                                                                                               Day_Hours                               Notes
                            East New York Farms: St. John Community Garden                                                                          474 New Jersey Ave, Brooklyn, NY 11207                                                                          